# emb_cos

## 导包

In [1]:
# import packages
import time, math, os
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
warnings.filterwarnings('ignore')

In [2]:
data_path = './data_raw/'
save_path = './tmp_results/'

## df节省内存函数

In [3]:
# 节约内存的一个标配函数
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

## 读取采样或全量数据

In [4]:
# debug模式：从训练集中划出一部分数据来调试代码
def get_all_click_sample(data_path, sample_nums=10000):
    """
        训练集中采样一部分数据调试
        data_path: 原数据的存储路径
        sample_nums: 采样数目（这里由于机器的内存限制，可以采样用户做）
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

# 读取点击数据，这里分成线上和线下，如果是为了获取线上提交结果应该讲测试集中的点击数据合并到总的数据中
# 如果是为了线下验证模型的有效性或者特征的有效性，可以只使用训练集
def get_all_click_df(data_path='./data_raw/', offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')

        all_click = trn_click.append(tst_click)
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

In [5]:
# 全量训练集
all_click_df = get_all_click_df(offline=False)

all_click_df.shape

(1630633, 9)

In [6]:
all_click_df.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


## 获取 用户 - 文章 - 点击时间字典

In [7]:
# 根据点击时间获取用户的点击文章序列   {user1: {item1: time1, item2: time2..}...}
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

## 获取点击最多的topk个文章

In [8]:
# 获取近期点击最多的文章
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

## itemcf的物品相似度计算

In [11]:
df_articles_emb = pd.read_csv(data_path + 'articles_emb.csv')
articleID2emb = {}

for _, row in df_articles_emb.iterrows():
    articleID2emb[row['article_id']] = row[1:].tolist()
    
len(articleID2emb)

364047

In [12]:
articleID2emb[0]

[-0.16118301,
 -0.95723313,
 -0.13794445,
 0.050855342000000005,
 0.83005524,
 0.90136534,
 -0.33514765,
 -0.55956066,
 -0.50060284,
 0.16518293,
 0.42843420000000004,
 0.35505559999999997,
 0.87443674,
 -0.52888286,
 0.6254871999999999,
 0.2689198,
 -0.8228351,
 -0.703853,
 -0.62584543,
 -0.15285493,
 -0.6662412,
 0.043294865999999994,
 0.1786375,
 0.046890084000000005,
 0.5945311,
 -0.18334764,
 0.19510683,
 -0.46763963,
 -0.30480695,
 0.35317516,
 0.27818817,
 0.5386231,
 -0.37120935,
 0.48989806,
 -0.103832886,
 0.11917368,
 0.13243659,
 -0.62108386,
 -0.45331132,
 0.34662652,
 -0.06174106,
 -0.7305939,
 -0.38411567,
 -0.94075835,
 0.06134219,
 0.4825816,
 0.28968322,
 -0.62269634,
 -0.050004438,
 0.42151213,
 -0.24257636,
 0.6687105,
 -0.509004,
 -0.46179956,
 0.043901782,
 0.28848746,
 0.4498246,
 -0.28486234,
 0.9167290000000001,
 0.70312876,
 0.85167396,
 -0.6272441,
 0.35773164,
 0.3901894,
 0.65293527,
 0.1036527,
 0.79829276,
 -0.10254639,
 -0.20451429999999998,
 0.37861153,

In [13]:
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a 
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

In [14]:
def itemcf_sim(df):
    """
        文章与文章之间的相似性矩阵计算
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        思路: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for i, i_click_time in item_time_list:
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0)
                
                i2i_sim[i][j] = cos_sim(articleID2emb[i], articleID2emb[j])
                
#     i2i_sim_ = i2i_sim.copy()
#     for i, related_items in i2i_sim.items():
#         for j, wij in related_items.items():
#             i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim, open(save_path + 'itemcf_i2i_sim_cos.pkl', 'wb'))
    
    return i2i_sim

In [15]:
i2i_sim = itemcf_sim(all_click_df)

len(i2i_sim)

100%|██████████| 250000/250000 [22:08<00:00, 188.18it/s] 


35380

In [16]:
i2i_sim[160417]

{156618: 0.8004270316627884,
 156560: 0.8651674797564272,
 160974: 0.8210493650849671,
 234813: 0.6119214268560986,
 300082: 0.5359304027992569,
 315105: 0.6394719684112893,
 336380: 0.4863696515044891,
 337143: 0.5756515567462133,
 199474: 0.6036881960363322,
 162300: 0.8345303600011079,
 158536: 0.8224164527700863,
 158906: 0.8135402759270995,
 158082: 0.7768125948776384,
 29997: 0.5524301136280645,
 158882: 0.7143053185503017,
 293513: 0.45800356579001583,
 119102: 0.425201769862091,
 168637: 0.7672578579880778,
 235689: 0.559570944802407,
 162655: 0.7929523695415803,
 156808: 0.8379562131797185,
 157076: 0.7225724732091069,
 161149: 0.6688081270748294,
 162663: 0.7972351413497929,
 161191: 0.732470464196738,
 354086: 0.48795645914558383,
 283627: 0.614004369960356,
 284985: 0.6218969440675653,
 162809: 0.7797246820360961,
 124176: 0.4585020764480857,
 233887: 0.5407938075722695,
 234446: 0.5396699814857567,
 203890: 0.7094268643397389,
 226639: 0.5435428561058884,
 235451: 0.601356

## itemcf 的文章推荐

In [17]:
# 基于商品的召回i2i
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: {item1: time1, item2: time2..}...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全        
        return: 召回的文章列表 {item1:score1, item2: score2...}
        注意: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items:
                continue
                
            item_rank.setdefault(j, 0)
            item_rank[j] +=  wij
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

## 给每个用户根据物品的协同过滤推荐文章

In [18]:
# 定义
user_recall_items_dict = defaultdict(dict)

# 获取 用户 - 文章 - 点击时间的字典
user_item_time_dict = get_user_item_time(all_click_df)

# 去取文章相似度
i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim_cos.pkl', 'rb'))

# 相似文章的数量
sim_item_topk = 10

# 召回文章数量
recall_item_num = 10

# 用户热度补全
item_topk_click = get_item_topk_click(all_click_df, k=50)

for user in tqdm(all_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, 
                                                        sim_item_topk, recall_item_num, item_topk_click)

100%|██████████| 250000/250000 [40:43<00:00, 102.32it/s]


## 召回字典转换成df

In [19]:
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])

100%|██████████| 250000/250000 [00:05<00:00, 43726.52it/s]


## 生成提交文件

In [20]:
# 生成提交文件
def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = save_path + model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)

In [21]:
# 获取测试集
tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
tst_users = tst_click['user_id'].unique()

# 从所有的召回数据中将测试集中的用户选出来
tst_recall = recall_df[recall_df['user_id'].isin(tst_users)]

# 生成提交文件
submit(tst_recall, topk=5, model_name='itemcf_emb_cos')

# 总结
本节内容主要包括赛题简介，数据概况，评价方式以及对该赛题进行了一个总体上的思路分析，作为竞赛前的预热，旨在帮助学习者们能够更好切入该赛题，为后面的学习内容打下一个良好的基础。最后我们给出了关于本赛题的一个简易Baseline， 帮助学习者们先了解一下新闻推荐比赛的一个整理流程， 接下来我们就对于流程中的每个步骤进行详细的介绍。

今天的学习比较简单，下面整理一下关于赛题理解的一些经验：

* 赛题理解究竟是在理解什么? 

>**理解赛题**：从直观上对问题进行梳理， 分析问题的目标，到底要让做什么事情, **这个非常重要**
>
>**理解数据**：对赛题数据有一个初步了解，知道和任务相关的数据字段和数据字段的类型， 数据之间的内在关联等，大体梳理一下哪些数据会对我们解决问题非常有用，方便后面我们的数据分析和特征工程。
>
>**理解评估指标**：评估指标是检验我们提出的方法，我们给出结果好坏的标准，只有正确的理解了评估指标，我们才能进行更好的训练模型，更好的进行预测。此外，很多情况下，线上验证是有一定的时间和次数限制的，**所以在比赛中构建一个合理的本地的验证集和验证的评价指标是很关键的步骤，能有效的节省很多时间**。 不同的指标对于同样的预测结果是具有误差敏感的差异性的所以不同的评价指标会影响后续一些预测的侧重点。

* 有了赛题理解之后，我们该做什么？

  >在对于赛题有了一定的了解后，分析清楚了问题的类型性质和对于数据理解 的这一基础上，我们可以梳理一个解决赛题的一个大题思路和框架
  >
  >我们至少要有一些相应的理解分析，比如**这题的难点可能在哪里，关键点可能在哪里，哪些地方可以挖掘更好的特征**.
  >
  >用什么样得线下验证方式更为稳定，**出现了过拟合或者其他问题，估摸可以用什么方法去解决这些问题**

  这时是在一个宏观的大体下分析的，有助于摸清整个题的思路脉络，以及后续的分析方向

**关于Datawhale：** Datawhale是一个专注于数据科学与AI领域的开源组织，汇集了众多领域院校和知名企业的优秀学习者，聚合了一群有开源精神和探索精神的团队成员。Datawhale 以“for the learner，和学习者一起成长”为愿景，鼓励真实地展现自我、开放包容、互信互助、敢于试错和勇于担当。同时 Datawhale 用开源的理念去探索开源内容、开源学习和开源方案，赋能人才培养，助力人才成长，建立起人与人，人与知识，人与企业和人与未来的联结。 本次数据挖掘路径学习，专题知识将在天池分享，详情可关注Datawhale：

![image-20201119112159065](http://ryluo.oss-cn-chengdu.aliyuncs.com/abc/image-20201119112159065.png)